In [1]:
import pandas as pd
import pymysql
import sqlalchemy
from sqlalchemy import create_engine, types
from datetime import datetime, date, time


In [ ]:
#连接本地mysql数据库
conn=pymysql.connect(
    host='localhost',
    user='root',
    password='',
    db='taobao_userbe',
    charset='utf8mb4')

In [2]:
df=pd.read_csv('UserBehavior.csv',header=None, names=['user_id', 'item_id', 'category_id','behavior_type', 'timestamp'])
df.drop_duplicates(subset=['user_id', 'item_id', 'category_id', 'behavior_type', 'timestamp'], inplace=True)#去重
df['behavior_datetime'] = pd.to_datetime(df['timestamp'], unit='s',utc=True).dt.tz_convert('Asia/Shanghai')
df['behavior_date']= df['behavior_datetime'].dt.date
df.drop(columns='timestamp', inplace=True)
df = df[(df['behavior_date'] >= date(2017, 11, 25)) & (df['behavior_date'] < date(2017, 12, 3))]
# print(len(df))
# print(df.head(5))
print(len(df))
df.head(5)

86433026


,user_id,item_id,category_id,behavior_type,behavior_datetime,behavior_date
0,1,2268318,2520377,pv,2017-11-25 01:21:10+08:00,2017-11-25
1,1,2333346,2520771,pv,2017-11-25 06:15:33+08:00,2017-11-25
2,1,2576651,149192,pv,2017-11-25 09:21:25+08:00,2017-11-25
3,1,3830808,4181361,pv,2017-11-25 15:04:53+08:00,2017-11-25
4,1,4365585,2520377,pv,2017-11-25 15:49:06+08:00,2017-11-25


In [11]:
user_active=df.groupby(by=['behavior_date','behavior_type'])['user_id'].count().reset_index().rename(columns={'user_id': 'behavior_count'})
print(user_active)

   behavior_date behavior_type  behavior_count
0     2017-11-25           buy          201144
1     2017-11-25          cart          563376
2     2017-11-25           fav          302071
3     2017-11-25            pv         9353416
4     2017-11-26           buy          205638
5     2017-11-26          cart          582581
6     2017-11-26           fav          308954
7     2017-11-26            pv         9567422
8     2017-11-27           buy          226834
9     2017-11-27          cart          541904
10    2017-11-27           fav          291221
11    2017-11-27            pv         9041186
12    2017-11-28           buy          211997
13    2017-11-28          cart          534157
14    2017-11-28           fav          289100
15    2017-11-28            pv         8842932
16    2017-11-29           buy          223068
17    2017-11-29          cart          551593
18    2017-11-29           fav          298587
19    2017-11-29            pv         9210820
20    2017-11

In [ ]:
connection_string = 'mysql+pymysql://'
engine = sqlalchemy.create_engine(connection_string)

In [14]:
# 将数据写入MySQL数据库
user_active.to_sql(name='user_active_table', con=engine, if_exists='replace', index=False, dtype={'behavior_date':types.Date(),
                                                                                                  'behavior_type':types.String(6),
                                                                                                  'unique_users':types.Integer()},chunksize=1000)

32

In [ ]:
user_active_notpye=df.groupby(by=['behavior_date'])['user_id'].nunique().reset_index().rename(columns={'user_id': 'total_users'})
user_active_notpye

In [ ]:
# 将数据写入MySQL数据库
user_active_notpye.to_sql(name='user_active_notpye_table', con=engine, if_exists='replace', index=False, dtype={'behavior_date':types.Date(),
                                                                                                  'unique_users':types.Integer()},chunksize=1000)

In [ ]:
#整合在一起
df.groupby(by=['behavior_date'])['user_id'].nunique().reset_index().rename(columns={'user_id': 'total_users'}).to_sql(
    name='user_active_notpye_table', con=engine, if_exists='replace', index=False, dtype={'behavior_date':types.Date(),
                                                                                        'unique_users':types.Integer()},chunksize=1000)

8

In [ ]:
catagory_active=df.groupby(by=['behavior_date','category_id','behavior_type'])['user_id'].count().reset_index().rename(columns={'user_id': 'catagory_behavior_count'})
catagory_active

In [ ]:
# 将结果存入MySQL数据库
catagory_active.to_sql(name='catagory_active_table', con=engine, if_exists='replace', index=False, dtype={'behavior_date':types.Date(),
                                                                                                  'category_id':types.Integer(),
                                                                                                  'behavior_type':types.String(6),
                                                                                                  'catagory_behavior_count':types.Integer()},chunksize=1000)

In [16]:
#整合在一起
df.groupby(by=['behavior_date','category_id','behavior_type'])['user_id'].count().reset_index().rename(columns={'user_id': 'catagory_behavior_count'}).to_sql(
    name='catagory_active_table', con=engine, if_exists='replace', index=False, dtype={'behavior_date':types.Date(),
                                                                                        'category_id':types.Integer(),
                                                                                        'behavior_type':types.String(6),
                                                                                        'catagory_behavior_count':types.Integer()},chunksize=1000, method='multi')

189171

In [ ]:
item_active=df.groupby(by=['behavior_date','item_id','behavior_type'])['user_id'].count().reset_index().rename(columns={'user_id': 'item_behavior_count'})
item_active

In [ ]:
df.drop(columns='timestamp', inplace=True)
df.head(5)

In [5]:
df.drop(columns='timestamp', inplace=True)

C:\Users\FOGGY\AppData\Local\Temp\ipykernel_30472\2471418655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns='timestamp', inplace=True)


In [9]:

df['date_hour']=df['behavior_datetime'].dt.hour
df.head(5)

,user_id,item_id,category_id,behavior_type,behavior_datetime,behavior_date,date_hour
0,1,2268318,2520377,pv,2017-11-25 01:21:10+08:00,2017-11-25,1
1,1,2333346,2520771,pv,2017-11-25 06:15:33+08:00,2017-11-25,6
2,1,2576651,149192,pv,2017-11-25 09:21:25+08:00,2017-11-25,9
3,1,3830808,4181361,pv,2017-11-25 15:04:53+08:00,2017-11-25,15
4,1,4365585,2520377,pv,2017-11-25 15:49:06+08:00,2017-11-25,15


In [11]:

def get_time_period(hour):
    if 0<=hour < 6:
        return '0-6'
    elif 6<= hour < 12:
        return '6-12'
    elif 12<=hour < 18:
        return '12-18'
    else:
        return '18-24'
    
df['time_period'] = df['behavior_datetime'].dt.hour.apply(get_time_period)
df.head(5)

,user_id,item_id,category_id,behavior_type,behavior_datetime,behavior_date,date_hour,time_period
0,1,2268318,2520377,pv,2017-11-25 01:21:10+08:00,2017-11-25,1,0-6
1,1,2333346,2520771,pv,2017-11-25 06:15:33+08:00,2017-11-25,6,6-12
2,1,2576651,149192,pv,2017-11-25 09:21:25+08:00,2017-11-25,9,6-12
3,1,3830808,4181361,pv,2017-11-25 15:04:53+08:00,2017-11-25,15,12-18
4,1,4365585,2520377,pv,2017-11-25 15:49:06+08:00,2017-11-25,15,12-18


In [ ]:
df.to_csv('UserBehavior_processed.csv', index=False, encoding='utf-8')

In [ ]:
df.reset_index(drop=True, inplace=True)
df.head(5)

In [9]:
time_period_count=df.groupby(by=['behavior_date','time_period','behavior_type'])['user_id'].count().reset_index().rename(columns={'user_id': 'hour_behavior_count'})
time_period_count.head(15)

,behavior_date,time_period,behavior_type,hour_behavior_count
0,2017-11-25,0-6,buy,11587
1,2017-11-25,0-6,cart,45611
2,2017-11-25,0-6,fav,27481
3,2017-11-25,0-6,pv,767022
4,2017-11-25,12-18,buy,70416
5,2017-11-25,12-18,cart,166452
6,2017-11-25,12-18,fav,92337
7,2017-11-25,12-18,pv,2834515
8,2017-11-25,18-24,buy,72929
9,2017-11-25,18-24,cart,236499


In [12]:
time_period_count.to_sql(name='time_period_count_table_beijing', con=engine, if_exists='replace', index=False, dtype={'behavior_date':types.Date(),
                                                                                                    'time_period':types.String(6), 
                                                                                                    'behavior_type':types.String(6),
                                                                                                    'hour_behavior_count':types.Integer()}, chunksize=1000)

128

In [12]:
#整合在一起
df.groupby(by=['behavior_date','time_period','behavior_type'])['user_id'].count().reset_index().rename(columns={'user_id': 'hour_behavior_count'}).to_sql(
    name='time_period_count_table_beijing', con=engine, if_exists='replace', index=False, dtype={'behavior_date':types.Date(),
                                                                                                'time_period':types.String(6), 
                                                                                                'behavior_type':types.String(6),
                                                                                                'hour_behavior_count':types.Integer()}, chunksize=1000)

128

In [4]:
user_behavior_count=df.groupby(by=['user_id','behavior_date','behavior_type']).agg({
    'item_id': [('item_id_c','count'),
                ('unique_items_count', 'nunique')],
    'category_id': 'nunique'
}).reset_index().rename(columns={
    'item_id_c': 'user_behavior_count',
    'unique_items_count': 'unique_items_count',
    'category_id': 'unique_categories_count'
})
user_behavior_count.head(5)

user_id behavior_date behavior_type             item_id                     \
                                      user_behavior_count unique_items_count   
0       1    2017-11-25            pv                   6                  6   
1       1    2017-11-26            pv                   1                  1   
2       1    2017-11-27            pv                   8                  8   
3       1    2017-11-28            pv                   7                  6   
4       1    2017-11-29            pv                   8                  7   

  unique_categories_count  
                  nunique  
0                       5  
1                       1  
2                       7  
3                       4  
4                       3

In [7]:
#将数据写回mysql
user_behavior_count.to_sql(name='user_behavior_table', con=engine, if_exists='replace', index=False, dtype={
    'user_id': types.Integer(),
    'behavior_date': types.Date(),
    'behavior_type': types.String(6),
    'user_behavior_count': types.Integer(),
    'unique_items_count': types.Integer(),
    'unique_categories_count': types.Integer()
}, chunksize=1500)

9986002

In [17]:
#整合在一起
df.groupby(by=['user_id','behavior_date','behavior_type']).agg({
    'item_id': [('item_id_c','count'),
                ('unique_items_count', 'nunique')],
    'category_id': 'nunique'
}).reset_index().rename(columns={
    'item_id_c': 'user_behavior_count',
    'unique_items_count': 'unique_items_count',
    'category_id': 'unique_categories_count'
}).to_sql(name='user_behavior_table', con=engine, if_exists='replace', index=False, dtype={
    'user_id': types.Integer(),
    'behavior_date': types.Date(),
    'behavior_type': types.String(6),
    'user_behavior_count': types.Integer(),
    'unique_items_count': types.Integer(),
    'unique_categories_count': types.Integer()
}, chunksize=1500, method='multi')

KeyboardInterrupt: 

In [8]:
df.groupby(by=['behavior_date','item_id','behavior_type'])['user_id'].size().reset_index().rename(columns={'user_id': 'behavior_count'}).to_sql(
    name='item_behavior_table', 
    con=engine, 
    if_exists='replace', 
    index=False, 
    dtype={
        'behavior_date': types.Date(),
        'item_id': types.Integer(),
        'behavior_type': types.String(6),
        'behavior_count': types.Integer()
    }, 
    chunksize=1500
)

17926039

In [4]:
#统计周活跃用户
df['user_id'].nunique()  # 计算唯一用户数

987990

In [5]:
#制作新的用户行为模型
df1=df.groupby(by=['user_id','item_id','behavior_type'])['behavior_datetime'].count().reset_index().rename(columns={'behavior_datetime': 'behavior_count'})

In [7]:
df1.head(5)

,user_id,item_id,behavior_type,behavior_count
0,1,46259,pv,2
1,1,79715,pv,1
2,1,230380,pv,1
3,1,266784,pv,2
4,1,1305059,pv,1


In [10]:
df1.groupby(by=['item_id','behavior_type']).agg({
    'user_id': 'nunique',
    'behavior_count': 'sum'
}).reset_index().rename(columns={
    'user_id': 'unique_users_count',
    'behavior_count': 'total_behavior_count'
}).to_sql(name='item_behavior_summary', con=engine, if_exists='replace', index=False, dtype={
    'item_id': types.Integer(),
    'behavior_type': types.String(6),
    'unique_users_count': types.Integer(),
    'total_behavior_count': types.Integer()
}, chunksize=1500, method='multi')

6384985

In [3]:
df1 = df.sample(frac=0.001, random_state=1)  # 抽取10%的数据进行测试
df1.head(5)

,user_id,item_id,category_id,behavior_type,behavior_datetime,behavior_date
70047890,536655,4682482,2640118,pv,2017-11-29 09:42:48+08:00,2017-11-29
43490577,24590,1616969,2885642,pv,2017-11-27 14:15:11+08:00,2017-11-27
15995044,816681,1517267,4145813,pv,2017-12-02 10:16:12+08:00,2017-12-02
86934630,392815,2112555,4640311,buy,2017-11-27 09:27:39+08:00,2017-11-27
72861621,665996,2517390,1421739,fav,2017-12-01 00:45:08+08:00,2017-12-01


In [8]:
result=df1.groupby(['user_id', 'item_id', 'behavior_type']).size().unstack(fill_value=0).reset_index().rename(columns={
                'pv': 'pv_count',
                'cart': 'cart_count',
                'fav': 'fav_count',
                'buy': 'buy_count'
            })


In [16]:
df1[df1['user_id']==33810]

,user_id,item_id,category_id,behavior_type,behavior_datetime,behavior_date
65619930,33810,2975559,4801426,pv,2017-11-30 00:55:39+08:00,2017-11-30
65619992,33810,86256,4756105,pv,2017-12-02 10:19:32+08:00,2017-12-02
65619944,33810,319074,2735466,pv,2017-12-01 00:11:42+08:00,2017-12-01
65619811,33810,2975559,4801426,pv,2017-11-25 17:04:37+08:00,2017-11-25


In [15]:
result[result['pv_count']>1]

behavior_type,user_id,item_id,buy_count,cart_count,fav_count,pv_count
2879,33810,2975559,0,0,0,2
3905,46066,2332338,0,0,0,2
4037,48041,3066271,0,0,0,2
11233,134763,1187150,0,0,0,2
18323,217050,1258433,0,0,0,2
19568,231366,4127222,0,0,0,2
21905,259819,5088787,0,0,0,2
24116,285748,1079420,0,0,0,2
28509,337181,4786624,0,0,0,2
29851,352704,1561513,0,0,0,2


In [13]:
df2=df1.head(20)
df2

,user_id,item_id,category_id,behavior_type,behavior_datetime,behavior_date
70047890,536655,4682482,2640118,pv,2017-11-29 09:42:48+08:00,2017-11-29
43490577,24590,1616969,2885642,pv,2017-11-27 14:15:11+08:00,2017-11-27
15995044,816681,1517267,4145813,pv,2017-12-02 10:16:12+08:00,2017-12-02
86934630,392815,2112555,4640311,buy,2017-11-27 09:27:39+08:00,2017-11-27
72861621,665996,2517390,1421739,fav,2017-12-01 00:45:08+08:00,2017-12-01
96406896,82912,1275702,4756105,pv,2017-12-02 13:28:03+08:00,2017-12-02
93455097,692682,1574216,866301,buy,2017-11-30 17:38:53+08:00,2017-11-30
20475807,108086,4939586,3252144,pv,2017-12-02 16:08:54+08:00,2017-12-02
44383819,286428,1136035,2885642,pv,2017-11-29 19:22:31+08:00,2017-11-29
89571778,511835,4798198,5071267,pv,2017-12-01 20:49:46+08:00,2017-12-01


In [14]:
df2.groupby(['user_id', 'item_id', 'behavior_type']).size().unstack(fill_value=0).reset_index().rename(columns={
                'pv': 'pv_count',
                'cart': 'cart_count',
                'fav': 'fav_count',
                'buy': 'buy_count'
            })

behavior_type,user_id,item_id,buy_count,fav_count,pv_count
0,24590,1616969,0,0,1
1,82912,1275702,0,0,1
2,108086,4939586,0,0,1
3,179086,4965209,0,0,1
4,286428,1136035,0,0,1
5,392815,2112555,1,0,0
6,398454,2456609,0,0,1
7,427601,2940276,0,0,1
8,511835,4798198,0,0,1
9,536655,4682482,0,0,1


In [3]:
df=df.groupby(['user_id', 'item_id', 'behavior_type']).size().unstack(fill_value=0).reset_index().rename(columns={
                'pv': 'pv_count',
                'cart': 'cart_count',
                'fav': 'fav_count',
                'buy': 'buy_count'
            })
df.head(5)

behavior_type,user_id,item_id,buy_count,cart_count,fav_count,pv_count
0,1,46259,0,0,0,2
1,1,79715,0,0,0,1
2,1,230380,0,0,0,1
3,1,266784,0,0,0,2
4,1,1305059,0,0,0,1


In [4]:
len(df)

66052921

In [15]:
df[df['pv_count']>0].shape[0] #点击浏览

61713539

In [10]:
df[df['pv_count']==0].shape[0] #未点击浏览

4339382

In [11]:
df[(df['pv_count']>0) & (df['buy_count']==0) & (df['cart_count']==0) & (df['fav_count']==0)].shape[0] #点击-流失

57735233

In [7]:
df['pv_count'].sum()

np.int64(77423371)

In [20]:
df[(df['pv_count']>0) & (df['cart_count']>0) & (df['fav_count']==0)].shape[0] # 点击-加购（不收藏）

2147036

In [21]:
df[(df['pv_count']>0) & (df['cart_count']==0) & (df['fav_count']>0)].shape[0] #点击-收藏（不加购）

932096

In [14]:
df[(df['pv_count']>0) & (df['buy_count']>0) & (df['cart_count']==0) & (df['fav_count']==0)].shape[0] #点击-购买

831242

In [26]:
57735233+2147036+932096+831242+67932

61713539

In [ ]:
df[(df['pv_count']>0) & (df['buy_count']>0) & (df['cart_count']>0) & (df['fav_count']==0)].shape[0] #点击-加购（不收藏）-购买

213006

In [23]:
df[(df['pv_count']>0) & (df['cart_count']>0) & (df['fav_count']>0)].shape[0] #点击-收藏加购

67932

In [25]:
df[(df['pv_count']>0) & (df['buy_count']>0) & (df['cart_count']>0) & (df['fav_count']>0)].shape[0] #点击-收藏加购-购买

10933

In [24]:
df[(df['pv_count']>0) & (df['buy_count']>0) & (df['cart_count']==0) & (df['fav_count']>0)].shape[0] #点击-收藏（不加购）-购买

74624